# Coronavirus Map

#### Author: Chandana Karunaratne
#### Date: 28 April 2020
#### Description: This code generates an interactive map of coronavirus cases in Ontario, Canada, using daily-updated data from the Government of Ontario Data Catalogue.


In [1]:
print("Hello, friend")

Hello, friend


In [2]:
import numpy as np
import pandas as pd
from datetime import datetime
import plotly.express as px
import plotly.io as pio


In [3]:
# Download data from: https://data.ontario.ca/dataset/confirmed-positive-cases-of-covid-19-in-ontario

# Open file containing data and read into dataframe:

covid_df = pd.read_csv('filepath/filename.csv')


In [4]:
# Check length (number of rows) in data file:

len(covid_df)


15381

In [5]:
# Show first 5 rows of data:

covid_df.head(5)


,Row_ID,Accurate_Episode_Date,Age_Group,Client_Gender,Case_AcquisitionInfo,Outcome1,Reporting_PHU,Reporting_PHU_Address,Reporting_PHU_City,Reporting_PHU_Postal_Code,Reporting_PHU_Website,Reporting_PHU_Latitude,Reporting_PHU_Longitude
0,1,2020-01-22,50s,FEMALE,Travel-Related,Resolved,Toronto Public Health,"277 Victoria Street, 5th Floor",Toronto,M5B 1W2,www.toronto.ca/community-people/health-wellnes...,43.656591,-79.379358
1,2,2020-01-21,50s,MALE,Travel-Related,Resolved,Toronto Public Health,"277 Victoria Street, 5th Floor",Toronto,M5B 1W2,www.toronto.ca/community-people/health-wellnes...,43.656591,-79.379358
2,3,2020-01-24,20s,FEMALE,Travel-Related,Resolved,Middlesex-London Health Unit,50 King Street,London,N6A 5L7,www.healthunit.com,42.981468,-81.254016
3,4,2020-02-05,20s,FEMALE,Travel-Related,Resolved,Toronto Public Health,"277 Victoria Street, 5th Floor",Toronto,M5B 1W2,www.toronto.ca/community-people/health-wellnes...,43.656591,-79.379358
4,5,2020-02-16,60s,FEMALE,Travel-Related,Resolved,Toronto Public Health,"277 Victoria Street, 5th Floor",Toronto,M5B 1W2,www.toronto.ca/community-people/health-wellnes...,43.656591,-79.379358


In [6]:
# Group the data by 'Reporting_PHU' and then by 'Accurate_Episode_Date' and then convert this series to a dataframe:

covid_count_df = covid_df.groupby('Reporting_PHU')['Accurate_Episode_Date'].value_counts().to_frame()


In [7]:
# Rename the 'Accurate_Episode_Date' column to 'Count':

covid_count_df.rename(columns = {'Accurate_Episode_Date':'Count'}, inplace = True)


In [8]:
# Flatten the hierarchical column indexes:

covid_count_df.reset_index(inplace=True)


In [9]:
# Create a dataframe containing latitude and longitude for each PHU:

PHU_df = covid_df.groupby('Reporting_PHU').first().reset_index()


In [10]:
# Drop all columns except for 'Reporting_PHU', 'Reporting_PHU_Latitude', 'Reporting_PHU_Longitude':

PHU_df = PHU_df[['Reporting_PHU', 'Reporting_PHU_Latitude', 'Reporting_PHU_Longitude']]


In [11]:
# Merge 'covid_count_df' and 'PHU_df':

covid_final_df = covid_count_df.merge(PHU_df, how = 'left', left_on = 'Reporting_PHU', right_on = 'Reporting_PHU')


In [12]:
# Convert string values in 'Accurate_Episode_Date' to datetime (infer datetime format) values:

covid_final_df['Accurate_Episode_Date'] = pd.to_datetime(covid_final_df['Accurate_Episode_Date'], infer_datetime_format=True)


In [13]:
# Sort 'covid_final_df' dataframe by 'Accurate_Episode_Date' and then 'Reporting_PHU':

covid_final_df = covid_final_df.sort_values(["Accurate_Episode_Date", "Reporting_PHU"], ascending = (True, True))

# Reset index in dataframe:

covid_final_df = covid_final_df.reset_index(drop=True)


In [14]:
# Convert all datetime values to string values (only day and month) to enable plotly time-series plot:

covid_final_df['Accurate_Episode_Date'] = covid_final_df.Accurate_Episode_Date.dt.strftime('%d %b')


In [15]:
# Rename columns so that map is easier to understand:

covid_final_df.columns = ['Public Health Unit', 'Date', 'Number of Cases', 'PHU Latitude', 'PHU Longitude']


In [16]:
# Get today's date (to be used as part of chart title):

current_date = datetime.today().strftime('%d %B')


In [ ]:
# Specify all parameters for map visualization:

fig = px.scatter_mapbox(covid_final_df, 
                     size="Number of Cases",
                     hover_name="Public Health Unit",
                     animation_frame="Date",
                     lat = "PHU Latitude",
                     lon = "PHU Longitude",
                     color_discrete_sequence=["crimson"],
                     title = "Interactive Map of Coronavirus Cases in Ontario (1 January - %s 2020)" %current_date,
                     zoom=5, 
                     height=600)

# Use Open Street Map as base map:

fig.update_layout(mapbox_style="open-street-map")

# Adjust margins on top, left, and right:

fig.update_layout(margin={"r":150,"t":40,"l":150,"b":0})

# Center title and increase font size:

fig.update_layout(title={'x':0.5,'xanchor':'center','font':{'size':20}})

# Increase frame duration so that length of time on each frame is increased (in milliseconds)

fig.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = 200

# Increase transition duration so that length of time between frames is increased (in milliseconds)

fig.layout.updatemenus[0].buttons[0].args[1]['transition']['duration'] = 200

fig.show()


In [18]:
# Create html file featuring chart to use on webpage:

# Source: https://towardsdatascience.com/how-to-create-a-plotly-visualization-and-embed-it-on-websites-517c1a78568b

# Ensure that filename is 'index.html'
# Parameter 'auto_open' refers to whether the html file will automatically open in a new tab
# Parameter 'auto-play' refers to whether the html file (and animation) will automatically play when file is opened

pio.write_html(fig, file='filepath/index.html', auto_open=True, auto_play=False)


In [19]:
# Sources:
# 1) https://data.ontario.ca/dataset/confirmed-positive-cases-of-covid-19-in-ontario
# 2) https://towardsdatascience.com/how-to-create-a-plotly-visualization-and-embed-it-on-websites-517c1a78568b
# 3) https://plotly.com/python/scatter-plots-on-maps/
# 4) https://plotly.com/python/bubble-maps/
# 5) https://plotly.com/python-api-reference/generated/plotly.express.scatter_geo.html
# 6) https://plotly.com/python/mapbox-layers/
# 7) https://www.youtube.com/watch?v=RCUrpCpGZ5o&feature=youtu.be
# 8) https://github.com/Coding-with-Adam/Dash-by-Plotly/blob/master/Plotly_Graphs/Animated_Scatter/gender_ineq.py
# 9) https://github.com/plotly/plotly.py/pull/1447/commits/b8a731ea6e0d56203833c10e90926473ed783032
# 10) https://docs.python.org/3/library/datetime.html

